# Тестирование

In [1]:
import datetime
import re
from typing import Optional

In [2]:
import unittest
import doctest

In [7]:
class RuDateParser:

    def parse_numeric(self, date: str) -> Optional[datetime.datetime]: 
        """ 
        Парсит даты в формате dd-mm-yyyy

        >>> RuDateParser().parse_numeric('01-12-2010')
        datetime.datetime(2010, 12, 1, 0, 0)

        >>> RuDateParser().parse_numeric('01/12/2010')

        >>> RuDateParser().parse_numeric('не дата')
        """
        if not isinstance(date, str):
            raise TypeError
        # \d - digit, то же самое, что [0-9]
        if not re.match('\d{2}-\d{2}-\d{4}', date): 
            return None
        date_splitted = date.split('-')
        day = int(date_splitted[0])
        month = int(date_splitted[1])
        year = int(date_splitted[2])
        try:
            return datetime.datetime(day=day, month=month, year=year)
        except ValueError:
            return None

In [12]:
class RuDateParserTestCase(unittest.TestCase):
    def setUp(self):
        self.parser = RuDateParser()

    # тестируем поведение при правильных входных данных
    def test_parse_numeric_matching_string(self):
        self.assertEqual(datetime.datetime(day=12, month=1, year=2020), 
                         self.parser.parse_numeric('12-01-2020'))
        
    # тестируем поведение при вводе строки, не содержащей дату в нужном формате
    def test_parse_numeric_unmatching_string(self):
        self.assertEqual(None, self.parser.parse_numeric('12/01/2020'))
        
    # тестируем поведение при неправильном типе входных данных
    def test_parse_numeric_incorrect_input_type(self):
        self.assertRaises(TypeError, self.parser.parse_numeric, 123)
    
    # тестируем поведение при неправильных цифрах даты
    def test_parse_numeric_incorrect_date(self):
        self.assertEqual(None, self.parser.parse_numeric('32-41-2020'))

# запустить все тесты
if __name__ == '__main__':
#     unittest.main()  # если запускаем в нормальном месте
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # если запускаем в jupyter

....
----------------------------------------------------------------------
Ran 4 tests in 0.004s

OK


In [9]:
dp = RuDateParser()

In [10]:
dp.parse_numeric('46-21-2020')

**Задание:** 
1. Подумать, какой ввод сломает метод parse_numeric (он выдаст ошибку, но не поднятую нами TypeError), что мы не учли при написании метода
2. Исправить метод  
3. Дописать соответсвующий тест
4. Запустить тесты не в тетрадке

## Запуск тестов 

### командная строка

+ все тесты в проекте: ```python -m unittest discover ```
+ все тесты в модуле: ``` python -m unittest tests.test_something ```
+ все тесты в классе (тест-кейсе): 
``` python -m unittest tests.test_something.SomeTestCase ```
+ с измерением coverage: \
``` coverage run -m unittest ... ``` \
затем ``` coverage report -m ```


Флаг ***-m*** означает, что интерпретор найдет модуль/пакет с нужным именем и запустит его как  [***\_\_main\_\_***](https://docs.python.org/3/library/__main__.html#module-__main__), расширение (***.py***) писать не нужно ([документация](https://docs.python.org/3/using/cmdline.html#cmdoption-m)).

Чтобы unittest.discover правильно находил тесты, все папки с ними должны быть импортируемымми пакетами (packages), то есть иметь в себе \_\_init\_\_
https://stackoverflow.com/questions/5088960/python-m-unittest-discover-does-not-discover-tests

### PyCharm

+ правой кнопкой мыши на папку/файл/класс, выбрать run 'Unittest in ... (with Coverage)'
![](https://webdevblog.ru/wp-content/uploads/2019/04/py_run_test_folder.png)
+ или нажать на зеленую стрелку рядом с тем местом, которое хотим запустить

## Хорошие тесты

+ один юнит-тест проверяет только один фрагмент кода
+ все юнит-тесты работают независимо. все тесты можно запустить по отдельности и в любом порядке
+ должны проверяться все варианты развития событий и все побочные эффекты

In [1]:
# объект который хранит состояние
class MemoryPlus:
    
    def __init__(self):
        self._memory = []
    
    def plus_ten(self, number):
        self._memory.append(number)
        return number + 10
    
    @property
    def memory(self):
        return self._memory

In [2]:
# плохие юнит-тесты - не смотрим на побочные эффекты (добавилось ли значение в memory)
import unittest
class MemoryPlusTestCase(unittest.TestCase):
    
    def setUp(self):
        self.mem_plus = MemoryPlus()
    
    def test_plus_ten(self):
        result = self.mem_plus.plus_ten(1)
        assert result == 11

if __name__ == '__main__':
#     unittest.main()  # если запускаем в нормальном месте
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # если запускаем в jupyter        

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [5]:
# плохие юнит-тесты - test_show_memory зависит от результатов test_plus_ten, 
# отдельно не будет работать, в другом порядке не будет работать
# важное замечание - unittest запускает тесты в алфаитном порядке, а не в написанном
import unittest
class MemoryPlusTestCase(unittest.TestCase):
    
    mem_plus = MemoryPlus()
    
    def test_plus_ten(self):
        result = self.mem_plus.plus_ten(1)
        assert result == 11
        
    def test_show_memory(self):
        assert self.mem_plus.memory == [1]
        
if __name__ == '__main__':
#     unittest.main()  # если запускаем в нормальном месте
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # если запускаем в jupyter        

F.
FAIL: test_memory (__main__.MemoryPlusTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-5-e50c72b8a719>", line 14, in test_memory
    assert self.mem_plus.memory == [1]
AssertionError

----------------------------------------------------------------------
Ran 2 tests in 0.003s

FAILED (failures=1)


In [2]:
# плохие юнит-тесты (хотя не настолько плохие как предыдущие) - напихали все в один тест
# код не выполнится дальше первого проваленного assert 
# и не получится узнать сразу не работает только добаление в memory или все остальное тоже
import unittest
class MemoryPlusTestCase(unittest.TestCase):
    
    def setUp(self):
        self.mem_plus = MemoryPlus()
    
    def test_plus_ten(self):
        result = self.mem_plus.plus_ten(1)
        
        assert self.mem_plus.memory == [1]
        assert result == 11

if __name__ == '__main__':
#     unittest.main()  # если запускаем в нормальном месте
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # если запускаем в jupyter        

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [4]:
# нормальные юнит-тесты
import unittest
class MemoryPlusTestCase(unittest.TestCase):
    
    def setUp(self):
        self.mem_plus = MemoryPlus()
    
    def test_plus_ten(self):
        result = self.mem_plus.plus_ten(1)
        assert result == 11
        
    def test_memory(self):
        self.mem_plus.plus_ten(1)
        assert self.mem_plus.memory == [1]

if __name__ == '__main__':
#     unittest.main()  # если запускаем в нормальном месте
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # если запускаем в jupyter        

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


### Coverage

+  Значение coverage (покрытия кода тестами) - % строчек кода, которые были запущены во время выполнения тестов
+ 100% покрытие не гарантирует того, что тесты хорошие и достаточные
+ Но хорошие и достаточные тесты должны (в идеале) обеспечивать 100% покрытие
+ [Документация](https://coverage.readthedocs.io/en/coverage-5.4/)

In [ ]:
!pip install coverage

**Задание:**
1. Реализовать метод parse_natural (по аналогии с parse_numeric), сделать так, чтобы он не ломался при любом вводе. 
2. Запустить doctest и проверить, что все хорошо. 
2. Написать к нему тесты используя unittest. 
4. Добиться 100% покрытия кода тестами (или близкого к 100)

In [10]:
import datetime
import re
from typing import Optional

class RuDateParser:
    
    month_dict = {'января': 1, 'февраля': 2,  'марта': 3,  'апреля': 4,  'мая': 5}


    def parse_natural(self, date: str) -> Optional[datetime.datetime]: 
        """ 
        Парсит даты в формате dd MMMM yyyy на русском языке

        >>> RuDateParser().parse_natural('1 января 2010')
        datetime.datetime(2010, 1, 1, 0, 0)

        >>> RuDateParser().parse_natural('1 Января 2010')
        datetime.datetime(2010, 1, 1, 0, 0)

        >>> RuDateParser().parse_natural('01 января 2010')
        datetime.datetime(2010, 1, 1, 0, 0)
        
        >>> RuDateParser().parse_natural('не дата')
        """
        if not isinstance(date, str):
            raise TypeError
            
        date = date.lower()
        # \d - digit, то же самое, что [0-9]
        if not re.match('\d{1,2} \w+ \d{4}', date): 
            return None
        
        date_splitted = date.split()
        
        if date_splitted[1] not in self.month_dict:
            return None
        
        day = int(date_splitted[0])
        month = self.month_dict[date_splitted[1]]
        year = int(date_splitted[2])
        try:
            return datetime.datetime(day=day, month=month, year=year)
        except ValueError:
            return None

In [11]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=4)

In [ ]:
import unittest

class RuDateParserTestCase(unittest.TestCase):
    
    def setUp(self):
        self.parser = RuDateParser()
        
    def test_parse_natural_wrong_type(self):
        self.assertRaises(TypeError, self.parser.parse_natural, 123)
        
    def test_parse_natural_wrong_format(self):
        result = self.parser.parse_natural('12-янв-2020')
        assert result == None
        
    def test_parse_natural_good_date_lower(self):
        result = self.parser.parse_natural('12 января 2020')
        assert result == datetime.datetime(2020, 1, 12, 0, 0)
        
    def test_parse_natural_good_date_upper(self):
        result = self.parser.parse_natural('12 Января 2020')
        assert result == datetime.datetime(2020, 1, 12, 0, 0)
    

## Динамическая генерация тестов

+ Нужно сделать много однотипных тестов (одна и та же функция/метод и т.д.)
+ Отличаются только входное значение и ожидаемый результат
+ Лучше не писать кучу почти одинакового кода, а генерировать тесты динамически. 

In [ ]:
def parse_name(name):
    parts = name.strip().split()
    surname, name, patr = '', '', ''
    if len(parts) == 1:
        name = parts[0]
    elif len(parts) == 2:
        surname, name = parts[0], parts[1]
    elif len(parts) == 3:
        surname, name, patr = parts
    elif len(parts) > 3:
        surname, name, patr = parts[0], ' '.join(parts[1:-1]), parts[-1]
    return surname, name, patr

In [ ]:
import unittest
class ParseNameTestCase(unittest.TestCase):
    def test_one_word(self):
        self.assertEqual(('', 'Петр', ''), parse_name('Петр'))
    def test_two_words(self):
        self.assertEqual(('Петров', 'Петр', ''), parse_name('Петров Петр'))
    def test_three_words(self):
        self.assertEqual(('Петров', 'Петр', 'Петрович'), 
                         parse_name('Петров Петр Петрович'))
    def test_more_words(self):
        self.assertEqual(('Петрова', 'Анна Мария', 'Васильевна'), 
                         parse_name('Петрова Анна Мария Васильевна'))
    def test_no_words(self):
        self.assertEqual(('', '', ''), parse_name(''))
        
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

Cпособ 1 - просто в цикле:
+ все данные проверяются в одном тесте
+ метод прекращает работу после первой AssertionError
+ все что дальше не проверяется 

In [ ]:
import unittest
class ParseNameTestCase(unittest.TestCase):
    def test_valid_name_parsing(self):
        for parsed_name, name in [
            (('', 'Петр', ''), ('Петр')),
            (('Петров', 'Петр', ''), ('Петров Петр')),
            (('Петров', 'Петр', 'Петрович'), ('Петров Петр Петрович')),
            (('Петрова', 'Анна Мария', 'Васильевна'), ('Петрова Анна Мария Васильевна')),
            (('', '', ''), (''))]:
            self.assertEqual(parsed_name, parse_name(name))
        
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

Cпособ 2 (он лучше) - c помощью пакета ***parametErized*** (не путать с parameTRized без e):
+ на каждую пару генерируется отдельный независимый тест
+ точно понятно, что именно пошло не так
+ один упавший тест не влияет на все остальные

In [ ]:
!pip install parameterized
from parameterized import parameterized 

In [ ]:
import unittest
class ParseNameTestCase(unittest.TestCase):
    @parameterized.expand(
        [(('', 'Петр', ''), ('Петр')),
         (('Петров', 'Петр', ''), ('Петров Петр')),
         (('Петров', 'Петр', 'Петрович'), ('Петров Петр Петрович')),
         (('Петрова', 'Анна Мария', 'Васильевна'), ('Петрова Анна Мария Васильевна')),
         (('', '', ''), (''))])
    def test_valid_name_parsing(self, parsed_name, name):
        self.assertEqual(parsed_name, parse_name(name))
        
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

**Задание**:
   + переписать тесты для parse_numeric с использованием parameterized

In [ ]:
import datetime
import re
from typing import Optional

class RuDateParser:

    def parse_numeric(self, date): 
        """ 
        Парсит даты в формате dd-mm-yyyy

        >>> RuDateParser().parse_numeric('01-12-2010')
        datetime.datetime(2010, 12, 1, 0, 0)

        >>> RuDateParser().parse_numeric('01/12/2010')

        >>> RuDateParser().parse_numeric('не дата')
        """
        if not isinstance(date, str):
            raise TypeError
        # \d - digit, то же самое, что [0-9]
        elif not re.match('\d{2}-\d{2}-\d{4}', date): 
            return None
        date_splitted = date.split('-')
        day = int(date_splitted[0])
        month = int(date_splitted[1])
        year = int(date_splitted[2]) 
        try:
            return datetime.datetime(day=day, month=month, year=year)
        except ValueError:
            return None

In [ ]:
import unittest
class RuDateParserTestCase(unittest.TestCase):
    
    def setUp(self):
        self.parser = RuDateParser()
        
    def tearDown(self):
        pass

    # тестируем поведение при правильных входных данных
    def test_parse_numeric_matching_string(self):
        self.assertEqual(datetime.datetime(day=12, month=1, year=2020), 
                         self.parser.parse_numeric('12-01-2020'))
        
    # тестируем поведение при вводе строки, не содержащей дату в нужном формате
    def test_parse_numeric_unmatching_string(self):
        self.assertEqual(None, self.parser.parse_numeric('12/01/2020'))
        
    def test_parse_numeric_day_too_large(self):
        self.assertEqual(None, self.parser.parse_numeric('52-01-2020'))
        
    def test_parse_numeric_month_too_large(self):
        self.assertEqual(None, self.parser.parse_numeric('21-21-2020'))
        
    def test_parse_numeric_day_zero(self):
        self.assertEqual(None, self.parser.parse_numeric('00-01-2020'))
        
    def test_parse_numeric_month_zero(self):
        self.assertEqual(None, self.parser.parse_numeric('01-00-2020'))
        
    # тестируем поведение при неправильном типе входных данных
    def test_parse_numeric_incorrect_input_type(self):
        self.assertRaises(TypeError, self.parser.parse_numeric, 123)

# запустить все тесты
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # colab

## Что еще полезного есть в unittest

### Пропуск тестов
+ если какие-то тесты нужны/должны работать только при определенных условиях

In [ ]:
# # это не рабочий код, он для примера и пояснения
class MyTestCase(unittest.TestCase):
    @unittest.skip("demonstrating skipping")
    def test_nothing(self):
        self.fail("shouldn't happen")

    @unittest.skipIf(mylib.__version__ < (1, 3),
                     "not supported in this library version")
    def test_format(self):
        # Tests that work for only a certain version of the library.
        pass

    @unittest.skipUnless(sys.platform.startswith("win"), "requires Windows")
    def test_windows_support(self):
        # windows specific testing code
        pass

    def test_maybe_skipped(self):
        if not external_resource_available():
            self.skipTest("external resource not available")
        # test code that depends on the external resource
        pass

In [ ]:
import unittest
def parse_name(name):
    parts = name.strip().split()
    surname, name, patr = '', '', ''
    if len(parts) == 1:
        name = parts[0]
    elif len(parts) == 2:
        surname, name = parts[0], parts[1]
    elif len(parts) == 3:
        surname, name, patr = parts
    elif len(parts) > 3:
        surname, name, patr = parts[0], ' '.join(parts[1:-1]), parts[-1]
    return surname, name, patr

class ParseNameTestCase(unittest.TestCase):
    @parameterized.expand(
        [(('', 'Петр', ''), ('Петр')),
         (('Петров', 'Петр', ''), ('Петров Петр')),
         (('Петров', 'Петр', 'Петрович'), ('Петров Петр Петрович')),
         (('Петрова', 'Анна Мария', 'Васильевна'), ('Петрова Анна Мария Васильевна')),
         (('', '', ''), (''))])
    def test_valid_name_parsing(self, parsed_name, name):
        self.assertEqual(parsed_name, parse_name(name))
        
    @unittest.skip("this feature is not implemented yet")    
    def test_wrong_type(self):
        self.assertEqual(('', '', ''), parse_name(None))
        
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

## Структура проекта c тестами

+ все тесты в отдельной папке, название test или tests
+ лучше превратить в пакет - добавить \_\_init\_\_.py (можно пустой)
+ названия файлов с тестами начинаются с test_

```
├── project root directory      
   ├── main project directory
   │   ├── ...
   │   ├── ...
   │    
   └── tests
       ├── __init__.py
       ├── test_*.py
       └── test_*.py 

```